# FAKE NEWS CLASSIFIER

## About dataset:
```
(WELFake) is a dataset of 72,134 news articles with 35,028 real and 37,106 fake news. For this, authors merged four popular news datasets (i.e. Kaggle, McIntire, Reuters, BuzzFeed Political) to prevent over-fitting of classifiers and to provide more text data for better ML training.

Dataset contains four columns: Serial number (starting from 0); Title (about the text news heading); Text (about the news content); and Label (0 = fake and 1 = real).

There are 78098 data entries in csv file out of which only 72134 entries are accessed as per the data frame.

Published in:
IEEE Transactions on Computational Social Systems: pp. 1-13 (doi: 10.1109/TCSS.2021.3068519).
```

DATASET LINK: https://www.kaggle.com/datasets/saurabhshahane/fake-news-classification

In [26]:
import numpy as np
import nltk
import re
import pandas as pd 
import tensorflow as tf 
from nltk.corpus import stopwords
from tensorflow.keras.layers import Embedding 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM 
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split



In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

print("Path to dataset files:", path)

Path to dataset files: /home/anson/.cache/kagglehub/datasets/saurabhshahane/fake-news-classification/versions/77


In [3]:
# IMPORTING LIBRARIES

import pandas as pd 


In [40]:
df = pd.read_csv('WELFake_Dataset.csv')

In [41]:
df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [42]:
df.isnull().sum()


Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [43]:
df = df.dropna()

In [44]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [45]:
# Preparing dataset
X = df.drop(['Unnamed: 0','label'], axis=1)
y = df['label']

In [46]:
X.head()

,title,text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ..."
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will..."
5,About Time! Christian Group Sues Amazon and SP...,All we can say on this one is it s about time ...


In [47]:
y.head()

0    1
2    1
3    0
4    1
5    1
Name: label, dtype: int64

In [16]:
df['combined_text'] = df['title'] + " " + df['text']
X = df['combined_text']


In [17]:
X.head()

0    LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2    UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3    Bobby Jindal, raised Hindu, uses story of Chri...
4    SATAN 2: Russia unvelis an image of its terrif...
5    About Time! Christian Group Sues Amazon and SP...
Name: combined_text, dtype: object

In [18]:
type(X)

pandas.core.series.Series

In [19]:
type(y)

pandas.core.series.Series

In [20]:
X = pd.DataFrame(X)

In [21]:
X.rename(columns={'combined_text': 'text'}, inplace=True)


In [22]:
X

,text
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,SATAN 2: Russia unvelis an image of its terrif...
5,About Time! Christian Group Sues Amazon and SP...
...,...
72129,Russians steal research on Trump in hack of U....
72130,WATCH: Giuliani Demands That Democrats Apolog...
72131,Migrants Refuse To Leave Train At Refugee Camp...
72132,Trump tussle gives unpopular Mexican leader mu...


In [23]:
y = pd.DataFrame(y)

In [24]:
y

,label
0,1
2,1
3,0
4,1
5,1
...,...
72129,0
72130,1
72131,0
72132,0


In [27]:
vocabulary_size = 20000

In [28]:
# one-hot implementation
messages = X.copy()
messages.reset_index(inplace=True)


In [29]:
messages

,index,text
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
2,3,"Bobby Jindal, raised Hindu, uses story of Chri..."
3,4,SATAN 2: Russia unvelis an image of its terrif...
4,5,About Time! Christian Group Sues Amazon and SP...
...,...,...
71532,72129,Russians steal research on Trump in hack of U....
71533,72130,WATCH: Giuliani Demands That Democrats Apolog...
71534,72131,Migrants Refuse To Leave Train At Refugee Camp...
71535,72132,Trump tussle gives unpopular Mexican leader mu...


In [30]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/anson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [33]:
ps_ = PorterStemmer()
corpus =[]

for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps_.stem(word) for word in review if not word in stopwords.words('english') ]
    review = ' '.join(review)
    corpus.append(review)

KeyboardInterrupt: 